In [0]:
#Sharanya Hunasamranahalli Thotadarya


In [0]:
import numpy as np
import os
import tensorflow as tf
from time import time
from google.colab import drive

In [2]:
drive.mount('/content/drive/')
%cd 'drive/My Drive/Colab Notebooks'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks


In [3]:
!python prepare_data2.py shakespere.txt skp -m 500 -p '\n\n'


2020-06-01 14:12:49.419406: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 31023 sequences...
Longest sequence is 3094 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 500 characters...
29430 sequences remaining.
Longest remaining sequence has length 499.
Removing length-0 sequences...
29430 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences..

In [4]:
from prepare_data2 import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'A': 3, 'k': 4, 'v': 5, '\n': 6, 'I': 7, '[': 8, 'j': 9, 'h': 10, 'E': 11, 'l': 12, 'Q': 13, 'd': 14, 'U': 15, 'p': 16, 'i': 17, ' ': 18, ':': 19, 'Y': 20, '?': 21, 'M': 22, 'C': 23, "'": 24, 'V': 25, '&': 26, 'X': 27, '.': 28, 'L': 29, 'f': 30, 'x': 31, 'K': 32, 'F': 33, 'O': 34, 'n': 35, 'B': 36, 'H': 37, 'a': 38, 'z': 39, 'r': 40, 's': 41, 'm': 42, 'G': 43, '!': 44, 'y': 45, 'c': 46, 'g': 47, 'R': 48, 'N': 49, '3': 50, 'b': 51, 'q': 52, 'J': 53, 'W': 54, ';': 55, 'D': 56, 'Z': 57, 'e': 58, 'T': 59, ']': 60, 'w': 61, 'P': 62, 'o': 63, 'S': 64, 't': 65, ',': 66, 'u': 67, '-': 68, '$': 69, '<PAD>': 0, '<S>': 1, '</S>': 2}
70


In [0]:
training_data=data.padded_batch(128,padded_shapes=500,padding_values=0,drop_remainder=True)


In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, vocab_size,
    batch_input_shape=[128, None],embeddings_initializer='identity'),
  tf.keras.layers.SimpleRNN(512,
                       return_sequences=True,
                       stateful=True,
                       recurrent_initializer='glorot_uniform'),
  tf.keras.layers.Dense(vocab_size)
])

In [0]:
#loss_fn = tf.nn.sparse_softmax_cross_entropy_with_logits()
opt = tf.optimizers.Adam(0.001)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [10]:
#epoch=0
#while epoch<10:

for step,batch in enumerate(training_data):

  model.reset_states()
  with tf.GradientTape() as tape:
    #lbl_batch = tf.TensorArray(tf.int64, size=tf.shape(batch)[0])

    lbl_batch=[]
    a=[]
    loss=[]

    logits = model(batch)
    for sequence in batch:
      lbl_batch.append(tf.cast(tf.roll(sequence,-1,axis=0),tf.int64)) #true label of each sequence in a batch
      a.append(tf.cast(tf.math.count_nonzero(sequence)-1,tf.float32)) #counting the number of unpadded inputs per sequence
    mask=tf.cast(tf.sequence_mask(a,500),tf.float32) #masking the 
    xent = tf.nn.sparse_softmax_cross_entropy_with_logits(lbl_batch, logits)*mask
    for count,seq in enumerate(xent):
      loss.append((tf.reduce_sum(seq)/a[count])) #loss for each sequence
    losses=tf.reduce_mean(loss) # mean loss of a batch
    #print(len(loss))
    varis = model.trainable_variables
    grads = tape.gradient(losses, varis)     
    opt.apply_gradients(zip(grads, varis))    
    #train_acc_metric(lbl_batch, logits)
    print("Loss: {}".format(losses))
    #break
    #epoch+=1








Loss: 4.271780967712402
Loss: 4.172835350036621
Loss: 3.7557153701782227
Loss: 3.6132912635803223
Loss: 3.56003737449646
Loss: 3.4657931327819824
Loss: 3.4109530448913574
Loss: 3.3243017196655273
Loss: 3.421020269393921
Loss: 6.121110916137695
Loss: 3.355785369873047
Loss: 4.0444769859313965
Loss: 3.4334304332733154
Loss: 3.4969301223754883
Loss: 3.549482583999634
Loss: 3.5225777626037598
Loss: 3.4543986320495605
Loss: 3.4118452072143555
Loss: 3.4215445518493652
Loss: 3.5348477363586426
Loss: 3.3709161281585693
Loss: 3.3442187309265137
Loss: 3.297036647796631
Loss: 3.26204252243042
Loss: 3.353374719619751
Loss: 3.3177120685577393
Loss: 3.276268720626831
Loss: 3.2974233627319336
Loss: 3.7454733848571777
Loss: 4.771059989929199
Loss: 4.112093925476074
Loss: 4.183445930480957
Loss: 3.941173553466797
Loss: 3.757911443710327
Loss: 3.505460739135742
Loss: 3.3618757724761963
Loss: 3.339449167251587
Loss: 3.2966794967651367
Loss: 3.363373279571533
Loss: 3.299055337905884
Loss: 3.39329099655151

In [0]:
model.save_weights('my_weights.model')

In [0]:

model1 = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, vocab_size,
    batch_input_shape=[1,None],embeddings_initializer='identity'),
  tf.keras.layers.SimpleRNN(512,
                       return_sequences=True,
                       stateful=True,
                       recurrent_initializer='glorot_uniform'),
  tf.keras.layers.Dense(vocab_size)
])
model1.load_weights('my_weights.model')

model1.build(tf.TensorShape([1, None]))

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}

idx2char = np.array(vocab)

idx2char = idx2char.tolist()

In [17]:
# input_eval = [char2idx[s] for s in start_string] # For any other symbol


num_generate = 1000
start_string = '<S>'
#predicted_id= '<S>'
# Converting our start string to numbers (vectorizing)
input_eval = [char2idx[start_string]]
input_eval = tf.expand_dims(input_eval, 0)

# Empty string to store our results
text_generated = []

# Low temperatures results in more predictable text.
# Higher temperatures results in more surprising text.
# Experiment to find the best setting.
temperature = 1.0

# Here batch size == 1
model.reset_states()
#while predicted_id!='</S>':
for i in range(num_generate):
    predictions = model1(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    for key, value in idx2char.items(): 

      if input_eval == value:      
        predicted_id = key
    text_generated.append(predicted_id)
    if predicted_id=='</S>':
      break

      
    text_generated.append(predicted_id)
print((start_string + ''.join(text_generated)))



<S>jjDDUUJJZZWWIIFFTTUU[[[[MM::

AAUUKKGGHHQQEELLxxRREEBBUUKKPPPP<S><S>KKHHAAMM&&HHIIMM&&CCFFOOOOPPMMEEGGRRQQHHXXRRAA<PAD><PAD>GGZZXXNN<PAD><PAD>EE::GGCCOO&&MMNN[[AARRGGOOUU<PAD><PAD>FFVVMMFFMMAAEEHHEEMMNNMMRROO33GGNNTTDDZZNNPPWWMMEEEENNDDMMLLOO<PAD><PAD>HHNNSSFFMMXXBBTTUU[[EEFFNNOO<PAD><PAD>FFJJqqPPLL</S>
